#ETL código principal

In [289]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [290]:
pd.options.display.float_format = "{:.2f}".format

In [291]:
df = pd.read_csv('https://raw.githubusercontent.com/wesleyesantos/Techchanlleng/main/ExpVinho.csv', sep=';')

In [292]:
df.shape

(128, 108)

In [293]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Columns: 108 entries, Id to 2022.1
dtypes: int64(107), object(1)
memory usage: 108.1+ KB


In [294]:
df.columns

Index(['Id', 'País', '1970', '1970.1', '1971', '1971.1', '1972', '1972.1',
       '1973', '1973.1',
       ...
       '2018', '2018.1', '2019', '2019.1', '2020', '2020.1', '2021', '2021.1',
       '2022', '2022.1'],
      dtype='object', length=108)

In [295]:
coluna_1 = df.columns[:2]
coluna_2 = df.columns[78:]

In [296]:
colunas = coluna_1.append(coluna_2)
colunas[2:]

Index(['2008', '2008.1', '2009', '2009.1', '2010', '2010.1', '2011', '2011.1',
       '2012', '2012.1', '2013', '2013.1', '2014', '2014.1', '2015', '2015.1',
       '2016', '2016.1', '2017', '2017.1', '2018', '2018.1', '2019', '2019.1',
       '2020', '2020.1', '2021', '2021.1', '2022', '2022.1'],
      dtype='object')

In [297]:
df = df[colunas]

In [298]:
df = df.drop('Id',axis=1)

In [299]:
df = df.set_index('País')

In [300]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 128 entries, Afeganistão to Vietnã
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   2008    128 non-null    int64
 1   2008.1  128 non-null    int64
 2   2009    128 non-null    int64
 3   2009.1  128 non-null    int64
 4   2010    128 non-null    int64
 5   2010.1  128 non-null    int64
 6   2011    128 non-null    int64
 7   2011.1  128 non-null    int64
 8   2012    128 non-null    int64
 9   2012.1  128 non-null    int64
 10  2013    128 non-null    int64
 11  2013.1  128 non-null    int64
 12  2014    128 non-null    int64
 13  2014.1  128 non-null    int64
 14  2015    128 non-null    int64
 15  2015.1  128 non-null    int64
 16  2016    128 non-null    int64
 17  2016.1  128 non-null    int64
 18  2017    128 non-null    int64
 19  2017.1  128 non-null    int64
 20  2018    128 non-null    int64
 21  2018.1  128 non-null    int64
 22  2019    128 non-null    int64
 23  2019.1 

In [301]:
df = df.reset_index()

In [302]:
colunas_df = df.columns
colunas_df[:5]

Index(['País', '2008', '2008.1', '2009', '2009.1'], dtype='object')

In [303]:
valores_usd = []
quantidade_l = []

for i in range(0,len(colunas_df)):
    if colunas_df[i] == 'País':
        valores_usd.append(colunas_df[i])
        quantidade_l.append(colunas_df[i])
    elif '.1' in colunas_df[i]:
        valores_usd.append(colunas_df[i])
    else:
        quantidade_l.append(colunas_df[i])

In [304]:
df_valores = df[valores_usd]
df_qtd = df[quantidade_l]

In [305]:
df_valores.columns = df_valores.columns.str.replace('\.1','', regex=True)

In [306]:
df_valores_norm = df_valores.reset_index().melt(id_vars=['País'], value_vars=df_valores.columns)
df_valores_norm.columns = ['pais_destino','ano','valor_usd']
df_valores_norm

,pais_destino,ano,valor_usd
0,Afeganistão,2008,0
1,África do Sul,2008,0
2,"Alemanha, República Democrática",2008,429970
3,Angola,2008,71083
4,Anguilla,2008,0
...,...,...,...
1915,Tuvalu,2022,0
1916,Uruguai,2022,997367
1917,Vanuatu,2022,0
1918,Venezuela,2022,32351


In [307]:
df_qtd_norm = df_qtd.reset_index().melt(id_vars=['País'], value_vars=df_qtd.columns)
df_qtd_norm.columns = ['pais_destino','ano','litros']
df_qtd_norm

,pais_destino,ano,litros
0,Afeganistão,2008,0
1,África do Sul,2008,0
2,"Alemanha, República Democrática",2008,265742
3,Angola,2008,25721
4,Anguilla,2008,0
...,...,...,...
1915,Tuvalu,2022,0
1916,Uruguai,2022,637117
1917,Vanuatu,2022,0
1918,Venezuela,2022,23220


In [308]:
df_qtd_norm.sort_values(by='litros', ascending=False).head(10)

,pais_destino,ano,litros
233,Rússia,2009,21912914
1761,Paraguai,2021,6522527
105,Rússia,2008,6207658
745,Rússia,2013,5893291
1889,Paraguai,2022,5076670
617,Rússia,2012,4528176
1633,Paraguai,2020,3299013
1377,Paraguai,2018,3234168
1505,Paraguai,2019,2419537
1249,Paraguai,2017,2393468


In [309]:
df = df_qtd_norm.join(df_valores_norm['valor_usd'])

In [310]:
lista_paises = df['pais_destino'].unique()

In [311]:
correcoes = {
    'Alemanha, República Democrática': 'Alemanha',
    'Antilhas Holandesas':'Antilhas Holandesas,Países Baixos',
    'Aruba': 'Aruba,Países Baixos',
    'Bósnia-Herzegovina': 'Bósnia-Herzegovina,Bosnia e Herzegovina',
    'Cayman, Ilhas': 'Ilhas Cayman',
    'Chipre': 'Chipre',
    'Cocos (Keeling), Ilhas': 'Cocos (Keeling), Ilhas ,Austrália',
    'Congo':'Congo,Países Baixos',
    'Coreia, Republica Sul': 'Coreia do Sul',
    'Curaçao':'Curaçao,Países Baixos',
    'Emirados Arabes Unidos': 'Emirados Árabes Unidos',
    'Eslovaca, Republica': 'Eslováquia',
    'Gibraltar':'Gibraltar,Reino Unido',
    'Guiana Francesa': 'Guiana Francesa',
    'Guine Bissau': 'Guiné-Bissau',
    'Guine Equatorial': 'Guiné Equatorial',
    'Hong Kong': 'Hong Kong',
    'Ilha de Man': 'Ilha de Man',
    'Indonésia': 'Indonésia',
    'Irã': 'Irã',
    'Países Baixos': 'Países Baixos',
    'Singapura': 'Singapura',
    'Taiwan (FORMOSA)': 'Taiwan',
    'Tcheca, República': 'República Tcheca',
    'Trinidade Tobago': 'Trinidade e Tobago',
    'Guiana Francesa':'Guiana Francesa,França',
    'Hong Kong':'Hong Kong,China',
    'Taiwan (FORMOSA)':'Taiwan,China'
}

In [312]:
lista_paises_corrigida = [correcoes.get(pais, pais) for pais in lista_paises]

In [313]:
df['pais_destino'] = df['pais_destino'].replace(correcoes)

In [314]:
df['pais_origem'] = 'Brasil'

In [315]:
ordem_colunas = ['pais_origem','pais_destino','ano','litros','valor_usd']
df = df[ordem_colunas]

In [316]:
df = df[(df != 0).all(axis =1)]

In [317]:
df['dia'] = 1
df['mes'] = 1
df['data'] = pd.to_datetime(df[['ano', 'mes', 'dia']].astype(str).agg('-'.join, axis=1), format='%Y-%m-%d')

In [318]:
 df = df.drop(['ano', 'mes', 'dia'], axis =1)

# Dados Externos

## Tabela 1 - Economia

In [319]:
dolar = pd.read_csv('https://github.com/wesleyesantos/Techchalleng/raw/main/dolar.csv')

In [320]:
dolar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Data      193 non-null    object 
 1   Último    193 non-null    float64
 2   Abertura  193 non-null    float64
 3   Máxima    193 non-null    float64
 4   Mínima    193 non-null    float64
 5   Vol.      0 non-null      float64
 6   Var%      193 non-null    object 
dtypes: float64(5), object(2)
memory usage: 10.7+ KB


In [321]:
dolar = dolar.drop('Vol.', axis=1)

In [322]:
dolar['Data'] = pd.to_datetime(dolar['Data'])

In [323]:
dolar.replace('%','',inplace=True,regex=True)
dolar['Var%'] = (dolar['Var%']).astype('float64')
dolar = dolar.sort_values(by='Var%',ascending=False)

Limpeza da coluna vol que está vazia

In [324]:
dolar['Ano']= dolar['Data'].dt.year
dolar['meses'] = dolar['Data'].dt.month
dolar = dolar.sort_values(by=['Ano','meses'])

In [325]:
dolar['Último'] = dolar['Último'].astype('float64')

## Tabela 2 -  Inclusão

In [326]:
inclusao = pd.DataFrame({'classificacao': ['Empresárias', 'Sommelières', 'Estudantes', 'Outros'],
                         'quantidade': [77, 49, 21, 53]})

## Tabela 3 - Demografia

In [327]:
populacao = pd.DataFrame([['Rússia', 2022, 143555736, 78.2],
                       ['Rússia', 2021, 143449286, 78.2],
                       ['Rússia', 2020, 144073139, 78.2],
                       ['Rússia', 2019, 144406261, 78.2],
                       ['Rússia', 2018, 144477859, 78.2],
                       ['Rússia', 2017, 144496739, 78.2],
                       ['Rússia', 2016, 144342397, 78.2],
                       ['Rússia', 2015, 144096870, 78.2],
                       ['Rússia', 2014, 143819667, 78.2],
                       ['Rússia', 2013, 143506995, 78.2],
                       ['Rússia', 2012, 143201721, 78.2],
                       ['Rússia', 2011, 142960908, 78.2],
                       ['Rússia', 2010, 142849468, 78.2],
                       ['Rússia', 2009, 142785349, 78.2],
                       ['Rússia', 2008, 142742366, 78.2],

                       ['Paraguai', 2022, 6780744, 62.3],
                       ['Paraguai', 2021, 6703799, 62.3],
                       ['Paraguai', 2020, 6618695, 62.3],
                       ['Paraguai', 2019, 6530026, 62.3],
                       ['Paraguai', 2018, 6443328, 62.3],
                       ['Paraguai', 2017, 6355404, 62.3],
                       ['Paraguai', 2016, 6266615, 62.3],
                       ['Paraguai', 2015, 6177950, 62.3],
                       ['Paraguai', 2014, 6090721, 62.3],
                       ['Paraguai', 2013, 6005652, 62.3],
                       ['Paraguai', 2012, 5923322, 62.3],
                       ['Paraguai', 2011, 5843939, 62.3],
                       ['Paraguai', 2010, 5768613, 62.3],
                       ['Paraguai', 2009, 5702574, 62.3],
                       ['Paraguai', 2008, 5645148, 62.3],

                       ['Estados Unidos', 2022, 333287557, 74],
                       ['Estados Unidos', 2021, 332031554, 74],
                       ['Estados Unidos', 2020, 331511512, 74],
                       ['Estados Unidos', 2019, 328329953, 74],
                       ['Estados Unidos', 2018, 326838199, 74],
                       ['Estados Unidos', 2017, 325122128, 74],
                       ['Estados Unidos', 2016, 323071755, 74],
                       ['Estados Unidos', 2015, 320738994, 74],
                       ['Estados Unidos', 2014, 318386329, 74],
                       ['Estados Unidos', 2013, 316059947, 74],
                       ['Estados Unidos', 2012, 313877662, 74],
                       ['Estados Unidos', 2011, 311583481, 74],
                       ['Estados Unidos', 2010, 309327143, 74],
                       ['Estados Unidos', 2009, 306771529, 74],
                       ['Estados Unidos', 2008, 304093966, 74],

                       ['China', 2022, 1412175000, 100],
                       ['China', 2021, 1412360000, 100],
                       ['China', 2020, 1411100000, 100],
                       ['China', 2019, 1407745000, 100],
                       ['China', 2018, 1402760000, 100],
                       ['China', 2017, 1396215000, 100],
                       ['China', 2016, 1387790000, 100],
                       ['China', 2015, 1379860000, 100],
                       ['China', 2014, 1371860000, 100],
                       ['China', 2013, 1363240000, 100],
                       ['China', 2012, 1354190000, 100],
                       ['China', 2011, 1345035000, 100],
                       ['China', 2010, 1337705000, 100],
                       ['China', 2009, 1331260000, 100],
                       ['China', 2008, 1324655000, 100],

                       ['Espanha', 2022, 47615034, 85],
                       ['Espanha', 2021, 47415750, 85],
                       ['Espanha', 2020, 47365655, 85],
                       ['Espanha', 2019, 47134837, 85],
                       ['Espanha', 2018, 46797754, 85],
                       ['Espanha', 2017, 46593236, 85],
                       ['Espanha', 2016, 46484062, 85],
                       ['Espanha', 2015, 46444832, 85],
                       ['Espanha', 2014, 46480882, 85],
                       ['Espanha', 2013, 46620045, 85],
                       ['Espanha', 2012, 46773055, 85],
                       ['Espanha', 2011, 46742697, 85],
                       ['Espanha', 2010, 46576897, 85],
                       ['Espanha', 2009, 46362946, 85],
                       ['Espanha', 2008, 45954106, 85],

                       ['Haiti', 2022, 11584996, 65.8],
                       ['Haiti', 2021, 11447569, 65.8],
                       ['Haiti', 2020, 11306801, 65.8],
                       ['Haiti', 2019, 11160438, 65.8],
                       ['Haiti', 2018, 11012421, 65.8],
                       ['Haiti', 2017, 10863543, 65.8],
                       ['Haiti', 2016, 10713849, 65.8],
                       ['Haiti', 2015, 10563757, 65.8],
                       ['Haiti', 2014, 10412740, 65.8],
                       ['Haiti', 2013, 10261206, 65.8],
                       ['Haiti', 2012, 10108539, 65.8],
                       ['Haiti', 2011, 9954312, 65.8],
                       ['Haiti', 2010, 9842880, 65.8],
                       ['Haiti', 2009, 9730638, 65.8],
                       ['Haiti', 2008, 9575247, 65.8],


                       ['Reino Unido', 2022, 66971411, 78.7],
                       ['Reino Unido', 2021, 67026300, 78.7],
                       ['Reino Unido', 2020, 67081234, 78.7],
                       ['Reino Unido', 2019, 66836327, 78.7],
                       ['Reino Unido', 2018, 66460344, 78.7],
                       ['Reino Unido', 2017, 66058859, 78.7],
                       ['Reino Unido', 2016, 65611593, 78.7],
                       ['Reino Unido', 2015, 65116219, 78.7],
                       ['Reino Unido', 2014, 64602298, 78.7],
                       ['Reino Unido', 2013, 64128273, 78.7],
                       ['Reino Unido', 2012, 63700215, 78.7],
                       ['Reino Unido', 2011, 63258810, 78.7],
                       ['Reino Unido', 2010, 62766365, 78.7],
                       ['Reino Unido', 2009, 62276270, 78.7],
                       ['Reino Unido', 2008, 61806995, 78.7],

                       ['Países Baixos', 2022, 17703090, 80.8],
                       ['Países Baixos', 2021, 17533044, 80.8],
                       ['Países Baixos', 2020, 17441500, 80.8],
                       ['Países Baixos', 2019, 17344874, 80.8],
                       ['Países Baixos', 2018, 17231624, 80.8],
                       ['Países Baixos', 2017, 17131296, 80.8],
                       ['Países Baixos', 2016, 17030314, 80.8],
                       ['Países Baixos', 2015, 16939923, 80.8],
                       ['Países Baixos', 2014, 16865008, 80.8],
                       ['Países Baixos', 2013, 16804432, 80.8],
                       ['Países Baixos', 2012, 16754962, 80.8],
                       ['Países Baixos', 2011, 16693074, 80.8],
                       ['Países Baixos', 2010, 16615394, 80.8],
                       ['Países Baixos', 2009, 16530388, 80.8],
                       ['Países Baixos', 2008, 16445593, 80.8],

                       ['Alemanha', 2022, 84079811, 84],
                       ['Alemanha', 2021, 83196078, 84],
                       ['Alemanha', 2020, 83160871, 84],
                       ['Alemanha', 2019, 83092962, 84],
                       ['Alemanha', 2018, 82905782, 84],
                       ['Alemanha', 2017, 82657002, 84],
                       ['Alemanha', 2016, 82348669, 84],
                       ['Alemanha', 2015, 81686611, 84],
                       ['Alemanha', 2014, 80982500, 84],
                       ['Alemanha', 2013, 80645605, 84],
                       ['Alemanha', 2012, 80425823, 84],
                       ['Alemanha', 2011, 80274983, 84],
                       ['Alemanha', 2010, 81776930, 84],
                       ['Alemanha', 2009, 81902307, 84],
                       ['Alemanha', 2008, 82110097, 84],

                       ['Japão', 2022, 125124989, 84],
                       ['Japão', 2021, 125681593, 84],
                       ['Japão', 2020, 126261000, 84],
                       ['Japão', 2019, 126633000, 84],
                       ['Japão', 2018, 126811000, 84],
                       ['Japão', 2017, 126972000, 84],
                       ['Japão', 2016, 127076000, 84],
                       ['Japão', 2015, 127141000, 84],
                       ['Japão', 2014, 127276000, 84],
                       ['Japão', 2013, 127445000, 84],
                       ['Japão', 2012, 127629000, 84],
                       ['Japão', 2011, 127833000, 84],
                       ['Japão', 2010, 128070000, 84],
                       ['Japão', 2009, 128047000, 84],
                       ['Japão', 2008, 128063000, 84]],
                       columns = ['pais_destino', 'ano', 'populacao', 'percentual_maior_idade']
                     )

In [328]:
populacao['dia'] = 1
populacao['mes'] = 1
populacao['data'] = pd.to_datetime(populacao[['ano', 'mes', 'dia']].astype(str).agg('-'.join, axis=1), format='%Y-%m-%d')

In [329]:
populacao = populacao.drop(['ano', 'mes', 'dia'],axis = 1 )

## Tabela 4 - Continentes

In [334]:
continentes = {
    'Afeganistão': 'Ásia',
    'África do Sul': 'África',
    'Alemanha': 'Europa',
    'Angola': 'África',
    'Antígua e Barbuda': 'América do Norte',
    'Antilhas Holandesas,Países Baixos': 'América Central',
    'Argentina': 'América do Sul',
    'Aruba,Países Baixos': 'América Central',
    'Austrália': 'Oceania',
    'Áustria': 'Europa',
    'Bahamas': 'América do Norte',
    'Bangladesh': 'Ásia',
    'Barbados': 'América do Norte',
    'Barein': 'Ásia',
    'Bélgica': 'Europa',
    'Belice': 'América Central',
    'Benin': 'África',
    'Bolívia': 'América do Sul',
    'Bósnia-Herzegovina,Bosnia e Herzegovina': 'Europa',
    'Brasil': 'América do Sul',
    'Bulgária': 'Europa',
    'Cabo Verde': 'África',
    'Camarões': 'África',
    'Canadá': 'América do Norte',
    'Catar': 'Ásia',
    'Ilhas Cayman': 'América Central',
    'Chile': 'América do Sul',
    'China': 'Ásia',
    'Chipre': 'Europa',
    'Cingapura': 'Ásia',
    'Cocos (Keeling), Ilhas ,Austrália': 'Oceania',
    'Colômbia': 'América do Sul',
    'Comores': 'África',
    'Congo,Países Baixos': 'África',
    'Coreia do Sul': 'Ásia',
    'Costa Rica': 'América Central',
    'Croácia': 'Europa',
    'Cuba': 'América do Norte',
    'Curaçao,Países Baixos': 'América Central',
    'Dinamarca': 'Europa',
    'Dominica': 'América do Norte',
    'El Salvador': 'América Central',
    'Emirados Árabes Unidos': 'Ásia',
    'Equador': 'América do Sul',
    'Eslováquia': 'Europa',
    'Espanha': 'Europa',
    'Estados Unidos': 'América do Norte',
    'Estônia': 'Europa',
    'Filipinas': 'Ásia',
    'Finlândia': 'Europa',
    'França': 'Europa',
    'Gana': 'África',
    'Gibraltar,Reino Unido': 'Europa',
    'Granada': 'América do Norte',
    'Grécia': 'Europa',
    'Guatemala': 'América Central',
    'Guiana': 'América do Sul',
    'Guiana Francesa,França': 'América do Sul',
    'Guiné-Bissau': 'África',
    'Guiné Equatorial': 'África',
    'Haiti': 'América do Norte',
    'Honduras': 'América Central',
    'Hong Kong,China': 'Ásia',
    'Hungria': 'Europa',
    'Ilha de Man': 'Europa',
    'India': 'Ásia',
    'Indonésia': 'Ásia',
    'Irã': 'Ásia',
    'Irlanda': 'Europa',
    'Itália': 'Europa',
    'Japão': 'Ásia',
    'Jordânia': 'Ásia',
    'Letônia': 'Europa',
    'Libéria': 'África',
    'Luxemburgo': 'Europa',
    'Malásia': 'Ásia',
    'Malta': 'Europa',
    'Marshall, Ilhas': 'Oceania',
    'Mauritânia': 'África',
    'México': 'América do Norte',
    'Moçambique': 'África',
    'Montenegro': 'Europa',
    'Namibia': 'África',
    'Nicaragua': 'América Central',
    'Nigéria': 'África',
    'Noruega': 'Europa',
    'Nova Caledônia': 'Oceania',
    'Nova Zelândia': 'Oceania',
    'Omã': 'Ásia',
    'Países Baixos': 'Europa',
    'Panamá': 'América Central',
    'Paraguai': 'América do Sul',
    'Peru': 'América do Sul',
    'Polônia': 'Europa',
    'Portugal': 'Europa',
    'Quênia': 'África',
    'Reino Unido': 'Europa',
    'Rússia': 'Europa',
    'São Tomé e Príncipe': 'África',
    'São Vicente e Granadinas': 'América do Norte',
    'Serra Leoa': 'África',
    'Singapura': 'Ásia',
    'Suazilândia': 'África',
    'Suécia': 'Europa',
    'Suíça': 'Europa',
    'Suriname': 'América do Sul',
    'Tailândia': 'Ásia',
    'Taiwan,China': 'Ásia',
    'República Tcheca': 'Europa',
    'Togo': 'África',
    'Trinidade e Tobago': 'América do Norte',
    'Turquia': 'Europa',
    'Tuvalu': 'Oceania',
    'Uruguai': 'América do Sul',
    'Vanuatu': 'Oceania',
    'Venezuela': 'América do Sul',
    'Vietnã': 'Ásia'
}

In [339]:
continentes = pd.DataFrame(list(continentes.items()), columns=['pais_destino', 'continentes'])